In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def ransac_image_alignment(image1, image2, num_iterations=1000, tolerance=5.0):
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # Match features using brute force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched keypoints
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])

    best_H = None
    max_inliers = 0

    for _ in range(num_iterations):
        # Randomly select 4 points
        indices = np.random.choice(len(matches), 4, replace=False)
        src_points = points1[indices]
        dst_points = points2[indices]

        # Compute homography using these points
        H, _ = cv2.findHomography(src_points, dst_points, cv2.RANSAC)

        if H is not None:
            # Warp image1 using the computed homography
            warped_image1 = cv2.warpPerspective(image1, H, (image2.shape[1], image2.shape[0]))

            # Calculate difference between warped_image1 and image2
            diff = cv2.absdiff(warped_image1, image2)
            gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
            _, mask = cv2.threshold(gray_diff, tolerance, 255, cv2.THRESH_BINARY_INV)
            inliers = cv2.countNonZero(mask)

            # Update best homography if the current one has more inliers
            if inliers > max_inliers:
                best_H = H
                max_inliers = inliers

    # Warp image1 using the best homography
    aligned_image = cv2.warpPerspective(image1, best_H, (image2.shape[1], image2.shape[0]))

    return aligned_image, best_H

# Align images using RANSAC
image1 = cv2.imread("/content/GlobalAlignment2.jpg")
image2 = cv2.imread("/content/GlobalAlignment1.jpg")
aligned_image, H = ransac_image_alignment(image1, image2)
cv2.imwrite("result.png", aligned_image)

# Display the result using cv2_imshow
cv2_imshow(aligned_image)

Output hidden; open in https://colab.research.google.com to view.